$\rm STEP1$ web上のzipファイルのダウンロード

In [1]:
import requests
import pandas as pd
import zipfile
import shapefile as shp
import matplotlib.pyplot as plt
import numpy as np
import time
from difflib import SequenceMatcher
import unicodedata
import re

In [2]:
df = pd.read_csv('code.csv')
code_list = list(df["tiiki-code"])

In [3]:
df['adress'] = df['ken-name']+df['sityouson-name1'].fillna('')+df['sityouson-name2'].fillna('')+df['sityouson-name3'].fillna('')
#df[df['ken-name'].str.contains('北海道')]
df2 = df[['adress']].copy()

e-Stat上にある都道府県別のzipファイルがあるURLを以下の関数で指定.

それぞれの市町村は総務省の定めた地域別コードが変数として定義されている.

In [207]:
def file_url(code):
    #地域コード
    code_name = "code=0"+str(code)
    #e-Statのそれぞれの地町村区のURL
    return "https://www.e-stat.go.jp/gis/statmap-search/data?dlserveyId=A002005212015&"+code_name+"&coordSys=1&format=shape&downloadType=5&datum=2000"

In [213]:
code_list[0]

1101

e-Stat上でダウンロードしたzip fileをローカルに保存する関数を以下に定める.

In [215]:
import urllib.request

def download_url(url, save_path):
    with urllib.request.urlopen(url) as dl_file:
        with open(save_path, 'wb') as out_file:
            out_file.write(dl_file.read())


#save_pathは自分の保存したいディレクトリに各自が変更して下さい.以下のディレクトリは作者の例.

#北海道 1~187(1-1694)
#save_path = "/Users/reisato/Desktop/jupyter-notebook/api/todohuken/hokkaido/0"+str(code_list[i])+".zip"
#東北
#save_path = "/Users/reisato/Desktop/jupyter-notebook/api/todohuken/tohoku/0"+str(code_list[i])+".zip"

start = time.time()
for i in range(0,187+1):
    save_path = "/Users/reisato/Desktop/jupyter-notebook/api/todohuken/hokkaido/0"+str(code_list[i])+".zip"
    url = file_url(code_list[i])
    download_url(url, save_path)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:363.3467390537262[sec]


zipファイルを別ファイルに解凍（shpファイルがzipファイルだと無理なので解凍する）

In [17]:
#zipファイルを別ファイルに解凍（shpファイルがzipファイルだと無理なので）
#北海道 2-209だがe-Statには1694までしかない
for i in range(205,209+1):
    with zipfile.ZipFile('todohuken/hokkaido/0'+str(code_list[i])+'.zip') as existing_zip:
        existing_zip.extractall('todohuken/hokkaido_kaito/0'+str(code_list[i]))

In [31]:
str(code_list[187])
#e-Stat 北海道:1695-1700は存在しない

'1694'

In [102]:
j = 11
sf = shp.Reader('todohuken/hokkaido_kaito/0'+str(code_list[j])+'/h27ka0'+str(code_list[j])+'.shp',encoding='cp932')
inf = sf.records()
inf#[0]#[5]

[Record #0: ['012030010', '01', '203', '001000', '北海道', '小樽市', '港町', '', 8101, 599583.676, 7130.655, 6844, 6843, '01', '北海道', '後志総合振興局', '小樽市', '', '0010', '-', '00', '203001000', '2030010', 'M', '', '', '', '', '港町', 8, 4, 2, 141.0086, 43.19724, '0010-00'],
 Record #1: ['012030020', '01', '203', '002000', '北海道', '小樽市', '堺町', '', 8101, 111600.919, 2211.009, 21001, 21000, '01', '北海道', '後志総合振興局', '小樽市', '', '0020', '-', '00', '203002000', '2030020', 'M', '', '', '', '', '堺町', 8, 67, 39, 141.0066, 43.19413, '0020-00'],
 Record #2: ['012030030', '01', '203', '003000', '北海道', '小樽市', '東雲町', '', 8101, 88903.739, 1358.906, 3411, 3410, '01', '北海道', '後志総合振興局', '小樽市', '', '0030', '-', '00', '203003000', '2030030', 'M', '', '', '', '', '東雲町', 12, 686, 419, 141.0039, 43.19397, '0030-00'],
 Record #3: ['012030040', '01', '203', '004000', '北海道', '小樽市', '相生町', '', 8101, 92987.034, 1553.864, 3777, 3776, '01', '北海道', '後志総合振興局', '小樽市', '', '0040', '-', '00', '203004000', '2030040', 'M', '', '', '', '', '

In [108]:
list(set(['小樽市小樽市']))

['小樽市小樽市']

In [49]:
pref = []
city = []
town = []
other = []
lon_min = []
lat_min = []
lon_max = []
lat_max = []
lon = []
lat = []
start = time.time()
for j in range(0,187+1):
    sf = shp.Reader('todohuken/hokkaido_kaito/0'+str(code_list[j])+'/h27ka0'+str(code_list[j])+'.shp',encoding='cp932')
    inf = sf.records()
    for i in range(0,len(sf.shapes())):
        pref.append(inf[i][4])
        city.append(inf[i][16]+inf[i][5])
        town.append(inf[i][6])
        other.append(sf.records()[i][7])
        #adress.append(sf.records()[0][4]+sf.records()[0][16]+sf.records()[0][5]+sf.records()[0][6]) #4:7の理由は以下のセルの中身の市区町村に該当する配列要素を取り出す
        lon_min.append(sf.shape(i).bbox[0])
        lat_min.append(sf.shape(i).bbox[1])
        lon_max.append(sf.shape(i).bbox[2])
        lat_max.append(sf.shape(i).bbox[3])
        
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:474.87265729904175[sec]


In [ ]:
#2015年北海道全域　elapsed_time:460.751886844635[sec]

In [42]:
i = 30
sf = shp.Reader('todohuken/hokkaido_kaito/0'+str(code_list[i])+'/h27ka0'+str(code_list[i])+'.shp',encoding='cp932')

In [43]:
sf.records()
print(sf.records()[i][4])
print(sf.records()[i][16]+sf.records()[i][5])
print(sf.records()[i][6])
print(sf.records()[i][7])

北海道
三笠市三笠市
幌内末広町



In [50]:
hokkaido = {
    'pref' : pref
    ,'city' : city
    ,'town' : town
    ,'other': other
    ,'lon_min' : lon_min
    ,'lat_min' : lat_min
    ,'lon_max' : lon_max
    ,'lat_max' : lat_max
}

In [49]:
hokkaido_table = pd.DataFrame.from_dict(hokkaido)
hokkaido_table.head(50)#[hokkaido_table['adress'].str.contains('北海道札幌市中央区旭ケ丘')]

,postal_code,adress
0,600000,北海道札幌市中央区以下に掲載がない場合
1,640941,北海道札幌市中央区旭ケ丘
2,600041,北海道札幌市中央区大通東
3,600042,北海道札幌市中央区大通西（１～１９丁目）
4,640820,北海道札幌市中央区大通西（２０～２８丁目）
5,600031,北海道札幌市中央区北一条東
6,600001,北海道札幌市中央区北一条西（１～１９丁目）
7,640821,北海道札幌市中央区北一条西（２０～２８丁目）
8,600032,北海道札幌市中央区北二条東
9,600002,北海道札幌市中央区北二条西（１～１９丁目）


In [92]:
s = '宮の森二条１３丁目'
re.split(r'(?<=条)', s)[0]

'宮の森二条'

In [ ]:
if hokkaido_table['town'][i].strs.contains('条'):
    re.split(r'(?<=条)', s)[0]
elif 

In [53]:
hokkaido_table.to_csv('Hokkaido_latlon.csv')

In [ ]:
groupby('adress',as_index=False).agg({'lon_min':np.min, 'lat_min':np.min,'lon_max':np.max,'lat_max':np.max})

In [13]:
from jusho import Jusho

In [204]:
postman = Jusho()
print(postman.from_postal_code('064-0951'))
aoba = postman.address_from_town('北海道','札幌市中央区', '宮の森一条','kanji')
print(aoba.hyphen_postal)

〒064-0951, 北海道 札幌市中央区 宮の森一条(HOKKAIDO SAPPORO SHI CHUO KU MIYANOMORI 1-JO)
064-0951


In [73]:
adress_code = []
start = time.time()
for i in range(0,len(hokkaido_table)):
    adress = postman.address_from_town(hokkaido_table['pref'][i],hokkaido_table['city'][i], hokkaido_table['town'][i],'kanji')
    if adress == [None]:
        adress_code.append(' ')
    else:
        adress_code.append(adress.hyphen_postal)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:1.7392091751098633[sec]


$\rm STEP2$
日本郵政の郵便コードと市区町村対応csvファイル(KEN_ALL_ROME.csv)をダウンロードする.

In [54]:
zenkoku = pd.read_csv('KEN_ALL_ROME.csv',encoding='cp932',header=None)
zenkoku = zenkoku.rename(columns={0:'postal_code', 1:'adress1' ,2:'adress2', 3:'adress3' ,4:'a' ,5:'b'})
#zenkoku['adress1'] = zenkoku['adress1'].str.cat(zenkoku['adress2'])
zenkoku['adress'] = zenkoku['adress1'].str.cat(zenkoku['adress2']).str.cat(zenkoku['adress3'])
zenkoku['adress'] = zenkoku['adress'].str.replace('\u3000','')
zenkoku = zenkoku[['postal_code','adress']]
pref = '北海道'
hokkaido = zenkoku[zenkoku['adress'].str.contains(pref)]
hokkaido = hokkaido[['postal_code','adress']]

In [110]:
latlon = pd.read_csv('Hokkaido_latlon.csv')
latlon['adress'] = latlon['pref']+latlon['city']+latlon['town']
latlon[latlon['adress'].str.contains('小樽市')]

,Unnamed: 0,pref,city,town,other,lon_min,lat_min,lon_max,lat_max,adress
6007,6007,北海道,小樽市小樽市,港町,NaN,141.000054,43.190866,141.015704,43.203754,北海道小樽市小樽市港町
6008,6008,北海道,小樽市小樽市,堺町,NaN,141.001549,43.190713,141.009224,43.197414,北海道小樽市小樽市堺町
6009,6009,北海道,小樽市小樽市,東雲町,NaN,141.001060,43.192296,141.006222,43.195577,北海道小樽市小樽市東雲町
6010,6010,北海道,小樽市小樽市,相生町,NaN,141.002738,43.189231,141.007037,43.193645,北海道小樽市小樽市相生町
6011,6011,北海道,小樽市小樽市,山田町,NaN,141.000185,43.190095,141.004183,43.195063,北海道小樽市小樽市山田町
...,...,...,...,...,...,...,...,...,...,...
6146,6146,北海道,小樽市小樽市,蘭島１丁目,NaN,140.844795,43.195341,140.865628,43.211936,北海道小樽市小樽市蘭島１丁目
6147,6147,北海道,小樽市小樽市,蘭島２丁目,NaN,140.845722,43.172851,140.873396,43.197697,北海道小樽市小樽市蘭島２丁目
6148,6148,北海道,小樽市小樽市,蘭島３丁目,NaN,140.857396,43.165267,140.889292,43.179254,北海道小樽市小樽市蘭島３丁目
6149,6149,北海道,小樽市小樽市,水面調査区,E1,141.001092,43.182151,141.035935,43.220202,北海道小樽市小樽市水面調査区


In [68]:
join_table = pd.merge(hokkaido, latlon, on='adress', how='left')
latlon_code = join_table[['postal_code','adress','lat_min','lon_min','lat_max','lon_max']]
(latlon_code[latlon_code['lat_min'].isnull()]).to_csv('not_join_hokaido_list.csv')

In [8]:
def adress_code(pref_city,adress):
    PREF = zenkoku[zenkoku['adress1'].str.contains(pref_city)]
    key = list(PREF['adress'])
    diff = []
    for i in list(PREF['adress']):
        diff.append(similarity1(unicodedata.normalize('NFKC', adress),unicodedata.normalize('NFKC', i)))
        #diff.append(similarity2(adress,i))
    return dict(zip(key,diff))#np.max(diff),adress[np.argmax(diff)]

In [9]:
test = adress_code('北海道札幌市中央区','北海道札幌市中央区大通西２１丁目')
test

NameError: name 'similarity1' is not defined

In [6]:
max(test, key=test.get)

NameError: name 'test' is not defined

In [182]:
pref = '北海道'
zenkoku[zenkoku['adress1'].str.contains(pref)]

,postal_code,adress1,adress
0,600000,北海道,北海道札幌市中央区以下に掲載がない場合
1,640941,北海道,北海道札幌市中央区旭ケ丘
2,600041,北海道,北海道札幌市中央区大通東
3,600042,北海道,北海道札幌市中央区大通西（１～１９丁目）
4,640820,北海道,北海道札幌市中央区大通西（２０～２８丁目）
...,...,...,...
8187,861821,北海道,北海道目梨郡羅臼町緑町
8188,861751,北海道,北海道目梨郡羅臼町峯浜町
8189,861841,北海道,北海道目梨郡羅臼町八木浜町
8190,861822,北海道,北海道目梨郡羅臼町湯ノ沢町
